# Prova de PLN

In [ ]:
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import os

import nltk
nltk.download('stopwords')

from nlputils import lexical

In [ ]:
#definição do diretorio dos corpus e criacao de uma lista com os nomes de cada arquivo dentro do diretorio
corpora_path = '../data/corpora/'
files_corpora = os.listdir(corpora_path)
files_corpora = [d for d in files_corpora if d not in '.DS_Store']

### Criação	de	uma	biblioteca	de	PLN,	com	rotinas	de	normalização	textual	do	nível	lexical:
- remoção	de	pontuação
- remoção	de	acentos
- remoção	de	stopwords
- lowercase
- stemming
- tokenizar	texto	em	sentenças
- tokenizer	texto	em	palavras

In [ ]:
import nltk
nltk.download('punkt')

import nltk
nltk.download('rslp')

In [ ]:
#chamada da bibioteca de preprocessamento
normalizer = lexical.Preprocessing()

### Cada	córpus	deve	estar	em	um	diretório	específico e	conter,	pelo	menos	500 textos	cada

In [ ]:
#criacao de um dicionario que ira armazenar cada corpus em uma chave
sentences_dic = {}
all_files = []
for corpus in files_corpora:
    files = [os.path.join(corpora_path + corpus, f) \
             for f in os.listdir(corpora_path + corpus) \
             if os.path.isfile(os.path.join(corpora_path + corpus, f))]
    #cada corpus tera mais 3 chaves para armazenar informacoes de trabalho
    sentences_dic[corpus] = {'sentencas': [], 'tokens': [], 'tamanho': []}
    
    #adiciona todos os arquivos em uma unica lista independentemente do corpus
    print(corpus)
    print(len(files))
    all_files.extend(files)
    
    #para cada arquivo em um corpus sera extraido suas frases e armazenadas em cada linha de uma lista
    for file in files:
        with open(file, 'r', encoding='utf-8') as text_file:
            lines = text_file.readlines()
            for line in lines:
                if line != '\n':
                    #armazenamento das sentencas do arquivo como escritas originalmente
                    sentences_dic[corpus]['sentencas'].append(line)

                    #toda a sentenca sera escrita em letras minusculas
                    line = normalizer.lowercase(line) 
                    #tokeniza as sentencas
                    sentences = normalizer.tokenize_sentences(line)
                    
                    #remove as pontuacoes
                    sentences = normalizer.remove_punctuation(sentences)
                    
                    #tokeniza as palavras de cada sentenca
                    sentences = [normalizer.tokenize_words(sent) for sent in sentences]
                    #print("numero sentencas:", len(sentences))
                    #remove os stopwords
                    sentences = normalizer.remove_stopwords(sentences)
                    
                    #armazena cada sentenca em forma de tokens
                    print(sentences)
                    sentences_dic[corpus]['tokens'].append(sentences[0])
                    #armazena o tamanho, em numero de palavras, de cada sentenca
                    sentences_dic[corpus]['tamanho'].append(len(sentences[0]))

In [ ]:
sentences_dic.keys()

In [ ]:
#utilizacao do Pandas para visualizao dos dados em forma de tabelas
import pandas as pd

#cracao de um dicionario que ira armazenar cada corpus em suas respectivas keys.
dataframes_dic = {}
for key in sentences_dic.keys():
    #os corpus armazenados aqui estara em formato de DataFrame onde cada key sera uma coluna da tabela
    dataframes_dic[key] = pd.DataFrame(sentences_dic[key], columns=['sentencas','tokens','tamanho'])

In [ ]:
dataframes_dic['animais'].head(n=10)

In [ ]:
dataframes_dic['games'].head(n=10)

In [ ]:
#criacao de uma "bag" de palavras dos tokens para cada corpus
dic_words = {}
for key in sentences_dic.keys():
    words_corpus = []
    for sentence in sentences_dic[key]['tokens']:
        no_stopwords_sentence = normalizer.remove_stopwords(sentence)
        words_corpus.extend(no_stopwords_sentence)
    #adiciona ao dicionario de palavras o corpus de tokens
    dic_words[key] = words_corpus

In [ ]:
#utilização do sklearn para facilitar a contagem de palavras com auxilio da bibioteca CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#função que realiza a contagem das palavras e retona 3 lista:
# - Lista de frenquencia de palavras(words_freq)
# - Lista das 20 palavras mais frequentes(most_freq_words)
# - Lista das 20 palavras menos frequentes(less_freq_words)
def words_frequency(token_corpus):
    vec = CountVectorizer().fit(token_corpus)

    #Here we get a Bag of Word model that has cleaned the text, removing non-aphanumeric characters and stop words.
    bag_of_words = vec.transform(token_corpus)

    #sum_words is a vector that contains the sum of each word occurrence in all texts in the corpus. 
    #In other words, we are adding the elements for each column of bag_of_words matrix.
    sum_words = bag_of_words.sum(axis=0)

    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    most_freq_words = sorted(words_freq, key = lambda x: x[1], reverse=True)
    less_freq_words = sorted(words_freq, key = lambda x: x[1])
    
    return words_freq, most_freq_words[:20], less_freq_words[:20]

In [ ]:
#dicionario que ira armazenar as inforções das palavras em cada corpus
word_analysis = {}

#dicionario de dataFrame das palavras listadas no dicionario word_analysis
word_df = {}

for key in dic_words.keys():
    #as listas retornadas pela função words_frequency é uma lista de lista que indica a palavra juntamente com sua
    #contagem de aparição no texto
    words_freq, most_freq_words, less_freq_words = words_frequency(dic_words[key])
    
    word_analysis[key] = {'Palavra': [],'Quantidade': [], 'Tamanho': []}
    
    #por palavra retornada da lista armazeno no dicionario separadamente a palavra, sua quantidade e seu tamanho
    for wd_qt in words_freq:
        word_analysis[key]['Palavra'].append(wd_qt[0])
        word_analysis[key]['Quantidade'].append(wd_qt[1])
        word_analysis[key]['Tamanho'].append(len(wd_qt[0]))

    word_df[key] = pd.DataFrame(word_analysis[key], columns=['Palavra','Quantidade','Tamanho'])

In [ ]:
word_df['animais'].head(n=10)

In [ ]:
word_df['games'].head(n=10)

### As	seguintes	estatísticas	devem ser	apresentadas, por	córpus:
- 20	palavras	mais	frequentes
- 20	palavras	menos	frequentes
- Tamanho	médio	das	palavras
- Tamanho	médio	das	sentenças,	em	número	de	palavras
- Outras	duas	estatísticas	que	achar	interessante

In [ ]:
#dicionario para armazenar as estatisticas sobre os corpus
corpus_analysis = {}

#dicionario que servirá para armazenar dataframes de estatisticas de cada corpus
analysis_df = {}
for corpus in sentences_dic.keys():
    #as listas retornadas pela função words_frequency é uma lista de lista que indica a palavra juntamente com sua
    #contagem de aparição no texto
    words_freq, mfw, lfw = words_frequency(dic_words[corpus])
    
    #dicionario de dicionario, onde cada corpus terá suas proprioas estatisticas armazenadas
    corpus_analysis[corpus] = {'Palavras Mais Frequentes(20)': [], 'Palavras Menos Frequentes(20)': [], 
                               'Tamanho medio das Palavras': [], 'Numero de sentencas': [], 
                               'Tamanho medio das sentencas': [], 'Word2Vec': []}
    
    #separa da lista de lista somente as palavra, armazena um vetor e salva no dicionario
    most_freq_words = []
    for w in mfw:
        most_freq_words.append(w[0])
    corpus_analysis[corpus]['Palavras Mais Frequentes(20)'].append(most_freq_words)
    
    #separa da lista de lista somente as palavra, armazena um vetor e salva no dicionario
    less_freq_words = []
    for w in lfw:
        less_freq_words.append(w[0])
    corpus_analysis[corpus]['Palavras Menos Frequentes(20)'].append(less_freq_words)
    
    #com uso do dataframe word_df é calculada a media do tamanho de todas as palavras do corpus adicionado ao
    #dicionario de estatistica
    words_mean = word_df[corpus]['Tamanho'].mean()
    corpus_analysis[corpus]['Tamanho medio das Palavras'].append(words_mean)
    
    #adiciona o numero de sentencas ao dicionario de estatisticas
    corpus_analysis[corpus]['Numero de sentencas'].append(len(sentences_dic[corpus]['sentencas']))
    
    #com uso do dataframe word_df é calculada a media do tamanho de todas as sentencas do corpus adicionado ao
    #dicionario de estatistica
    sentences_mean = dataframes_dic[corpus]['tamanho'].mean()
    corpus_analysis[corpus]['Tamanho medio das sentencas'].append(sentences_mean)
    
    #criacao do dataframe com os dados estatisticos adicinados ao dicinario corpus_analysis
    analysis_df[corpus] = pd.DataFrame(corpus_analysis[corpus], columns=['Palavras Mais Frequentes(20)',
                                                                        'Palavras Menos Frequentes(20)',
                                                                       'Tamanho medio das Palavras',
                                                                       'Numero de sentencas',
                                                                       'Tamanho medio das sentencas'])

In [ ]:
#exibe as estatisticas para todos os corpus
for corpus in corpus_analysis.keys():
    print("\nEstátisticas do corpus", corpus)
    print("20 palavras mais frequentes: \n", corpus_analysis[corpus]['Palavras Mais Frequentes(20)'])
    print("20 palavras menos frequentes: \n", corpus_analysis[corpus]['Palavras Menos Frequentes(20)'])
    print("Tamanho medio das Palavras: ", corpus_analysis[corpus]['Tamanho medio das Palavras'])
    print("Numero de sentencas: ", corpus_analysis[corpus]['Numero de sentencas'])
    print("Tamanho medio das sentencas: ", corpus_analysis[corpus]['Tamanho medio das sentencas'])
    

In [ ]:
#exibe o dataframe de estatisticas para os corpus
analysis_df['animais'].head()

In [ ]:
analysis_df['games'].head()

### Normalização	dos	corpora,	com	vistas	à	criação	de	dois	modelos,	a	saber,	Word2Vec	e	Doc2Vec.
- Para	cada	córpus, será criado	um	modelo	Word2Vec
- Para	todos	os	corpora,	apenas	um	modelo	Doc2Vec	será criado

#### Para	cada	córpus, será criado	um	modelo	Word2Vec

In [ ]:
#importacao das bibiotecas w2vec e d2vec
from gensim.models import Word2Vec, Doc2Vec

In [ ]:
#para cada corpus é adicionado em seu dicionario um modelo de word2vec 
for corpus in corpus_analysis.keys():
    sentence_tokens = sentences_dic[corpus]['tokens']
    w2vmodel = Word2Vec(sentences=sentence_tokens, size=300,min_count=5, workers=4, window=2)
    corpus_analysis[corpus]['Word2Vec'] = w2vmodel

#### Para todos os corpora, apenas um modelo Doc2Vec será criado

- preparação dos documentos em uma lista

In [ ]:
#lista que recebera cada documentos de todos os corpora em cada uma de suas posicoes
all_documents = []
for file in all_files:
    with open(file, 'r', encoding='utf-8') as text_file:
        document = ' '.join(text_file.readlines())
        document = normalizer.lowercase(document)
        document_tokens = normalizer.tokenize_words(document)
        all_documents.append(document_tokens)
print("Number of documents: {}".format(len(all_documents)))

#enumera os documentos taggeando-os e salvando-os em tagged_documents
tagged_documents = [TaggedDocument(words=d, tags=[str(i)]) for i, d in enumerate(all_documents)]

- treino do modelo de doc2vec para todos os arquivos do corpora

In [ ]:
#treina um modelo de doc2vec apartir da lista de documentos tageados
d2vmodel = Doc2Vec(tagged_documents, vector_size=20, window=2, min_count=1, workers=4)

### Uso	dos	modelos	gerados

- Uso	do	Word2Vec

#### Dada	uma	palavra w1 de	um	córpus,	quais	as	10	palavras	mais	similares a	w1?
1. Exemplifique	 com	 três	 palavras

In [ ]:
#define a quantidade de palavras similares deseja buscar
n_words = 10

#palavras para encontrar similaridade de acordo com os modelos treinados com o word2vec
test_word = [['água', 'alimentação', 'cuidados'],['jogos', 'jogadores', 'online']]

cont = 0
for corpus in corpus_analysis:
    print("\nCorpus", corpus)
    print("\nAs ", n_words, " palavras mais similares a ", test_word[cont][0], "sao:")
    print(corpus_analysis[corpus]['Word2Vec'].wv.most_similar(test_word[cont][0], topn=n_words))
    print("\nAs ", n_words, " palavras mais similares a ", test_word[cont][1], "sao:")
    print(corpus_analysis[corpus]['Word2Vec'].wv.most_similar(test_word[cont][1], topn=n_words))
    print("\nAs ", n_words, " palavras mais similares a ", test_word[cont][2], "sao:")
    print(corpus_analysis[corpus]['Word2Vec'].wv.most_similar(test_word[cont][2], topn=n_words))
    cont = cont+1

1. discuta	 como	 poderia	melhorar	 os	resultados.	Pense	no	nível	da	morfologia	ou	outro	do	PLN.

Para melhorar o resultado do word2vec é utilizar um corpus maior, com mais palavras, outros meios tambem é melhorar a normatização dos dados removendo stopwords, colocar as palavras em minusculas e utiliar uma boa fonte para extração dos corpus.

2. Tentar	alguma	abordagem	para	comparar	dois	documentos	diferentes	utilizando	os	vetores	do	Word2Vec

- Uso	do	Doc2Vec

#### Dados	os	documentos	(textos) de	corporas	diferentes,	utilize	os	vetores	para encontrar	os	documentos	mais	similares

1. Exemplifique	com	três	documentos	e	discuta	os resultados.	Ao	ser	ver, foram	bons?	Os	documentos	realmente	são	parecidos?	O	que	poderia	 ser	feito	para	melhorar os	resultados?

In [ ]:
#numero de documentos similares que deseja procurar
n_similar = 5

#posicao do documento na lista all_documentos que deseja procurar documentos similares
n_doc = 50
n_doc1 = 950
n_doc2 = 498

print("\ndocumentos similares ao documento: ", all_files[n_doc])
similar_docs = d2vmodel.docvecs.most_similar(n_doc, topn=n_similar)
for doc_num in similar_docs:
    num_doc = int(doc_num[0])
    print(all_files[num_doc])
    
print("\ndocumentos similares ao documento: ", all_files[n_doc1])
similar_docs = d2vmodel.docvecs.most_similar(n_doc1, topn=n_similar)
for doc_num in similar_docs:
    num_doc = int(doc_num[0])
    print(all_files[num_doc])

print("\ndocumentos similares ao documento: ", all_files[n_doc2])
similar_docs = d2vmodel.docvecs.most_similar(n_doc2, topn=n_similar)
for doc_num in similar_docs:
    num_doc = int(doc_num[0])
    print(all_files[num_doc])